In [1]:
!nvidia-smi

Sun Oct 27 09:39:02 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.104      Driver Version: 410.104      CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   74C    P0    31W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
!sudo pip3 install contractions
!sudo pip3 install textsearch
!sudo pip3 install tqdm
!sudo pip3 install nltk
!sudo pip3 install beautifulsoup4
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt

%matplotlib inline

# fix random seed for reproducibility
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

print("TF Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("TF Hub version: ", hub.__version__)
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

TF Version:  2.0.0
Eager mode:  True
TF Hub version:  0.6.0
GPU is available


In [4]:
dataset = pd.read_csv(r'https://github.com/dipanjanS/nlp_workshop_dhs18/raw/master/Unit%2011%20-%20Sentiment%20Analysis%20-%20Unsupervised%20Learning/movie_reviews.csv.bz2', compression='bz2')
dataset['sentiment'] = [1 if record == 'positive' else 0 for record in dataset['sentiment']]
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
review       50000 non-null object
sentiment    50000 non-null int64
dtypes: int64(1), object(1)
memory usage: 781.4+ KB


In [5]:
dataset.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [6]:
reviews = dataset['review'].values
sentiments = dataset['sentiment'].values

train_reviews = reviews[:5000]
val_reviews = reviews[5000:10000]
test_reviews = reviews[10000:]



train_sentiments = sentiments[:5000]
val_sentiments = sentiments[5000:10000]
test_sentiments = sentiments[10000:]

train_reviews.shape, val_reviews.shape, test_reviews.shape

((5000,), (5000,), (40000,))

In [7]:
import contractions
from bs4 import BeautifulSoup
import numpy as np
import re
import tqdm
import unicodedata


def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    [s.extract() for s in soup(['iframe', 'script'])]
    stripped_text = soup.get_text()
    stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
    return stripped_text

def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

def pre_process_corpus(docs):
    norm_docs = []
    for doc in tqdm.tqdm(docs):
        doc = strip_html_tags(doc)
        doc = doc.translate(doc.maketrans("\n\t\r", "   "))
        doc = doc.lower()
        doc = remove_accented_chars(doc)
        doc = contractions.fix(doc)
        # lower case and remove special characters\whitespaces
        doc = re.sub(r'[^a-zA-Z0-9\s]', ' ', doc, re.I|re.A)
        doc = re.sub(' +', ' ', doc)
        doc = doc.strip()  
        norm_docs.append(doc)
    return norm_docs

In [8]:
%%time

norm_train_texts = pre_process_corpus(train_reviews)
norm_val_texts = pre_process_corpus(val_reviews)
norm_test_texts = pre_process_corpus(test_reviews)

100%|██████████| 40000/40000 [00:19<00:00, 2070.11it/s]

CPU times: user 24.1 s, sys: 172 ms, total: 24.3 s
Wall time: 24.1 s


In [9]:
norm_train_texts = np.array(norm_train_texts)
norm_val_texts = np.array(norm_val_texts)
norm_test_texts = np.array(norm_test_texts)

In [10]:
norm_train_texts[:3]

array(['one of the other reviewers has mentioned that after watching just 1 oz episode you will be hooked they are right as this is exactly what happened with me the first thing that struck me about oz was its brutality and unflinching scenes of violence which set in right from the word go trust me this is not a show for the faint hearted or timid this show pulls no punches with regards to drugs sex or violence its is hardcore in the classic use of the word it is called oz as that is the nickname given to the oswald maximum security state penitentary it focuses mainly on emerald city an experimental section of the prison where all the cells have glass fronts and face inwards so privacy is not high on the agenda em city is home to many aryans muslims gangstas latinos christians italians irish and more so scuffles death stares dodgy dealings and shady agreements are never far away i would say the main appeal of the show is due to the fact that it goes where other shows would not dare for

In [11]:
model = "https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1"
hub_layer = hub.KerasLayer(model, output_shape=[128], input_shape=[], 
                           dtype=tf.string, trainable=True)
print(hub_layer(norm_train_texts[:3]))

tf.Tensor(
[[ 2.1637728   0.18443683  0.33868548  0.18714745  0.22127508  0.03885808
   0.20959312 -0.31841168 -0.2998969   0.06570792  0.29815578 -0.36403018
  -0.36856505 -0.36688802 -0.15620337  0.12879042 -0.5279176  -0.05068737
  -0.5405576   1.0036108   0.0983031   0.13588336 -0.24809034 -0.25477773
   0.10724348 -0.25031033  0.2962077  -0.00750021 -0.20377205  0.14243263
   0.07617488  0.1518155   0.17101593  0.02466883  0.36238325 -0.12907393
  -0.24922621 -0.41444957  0.10826241  0.50078815 -0.4214459  -0.1228352
  -0.12632468 -0.10329366  0.22902705  0.31700385  0.08330556  0.16459242
  -0.08031587  0.02723667  0.11583095  0.03190616 -0.06533278  0.31237805
  -0.24900854 -0.0130042   0.06682863 -0.26658732 -0.06602515  0.04475337
  -0.5149443   0.17992204  0.05797209 -0.40984443 -0.01934146  0.15096185
   0.17236525 -0.46626076  0.32511476 -0.0269111  -0.45234472  0.2429736
   0.00959222 -0.1835455  -0.10930699  0.40188032 -0.01052749 -0.5940521
  -0.18280017  0.141591    0.1

In [12]:
model = tf.keras.models.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.15))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.15))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adagrad', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 128)               124642688 
_________________________________________________________________
dense (Dense)                (None, 128)               16512     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 124,675,841
Trainable params: 124,675,841
Non-trainable params: 0
__________________________________________

In [13]:
EPOCHS=100
BATCH_SIZE=128

In [14]:
# Fit the model
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                      patience=3,
                                      restore_best_weights=True,
                                      verbose=1)
    
model.fit(norm_train_texts, train_sentiments, 
          validation_data=(norm_val_texts, val_sentiments),
          epochs=EPOCHS, 
          batch_size=BATCH_SIZE, 
          shuffle=True,
          callbacks=[es],
          verbose=1)

Train on 5000 samples, validate on 5000 samples
Epoch 1/100
5000/5000 [==============================] - 3s 581us/sample - loss: 0.6631 - accuracy: 0.6354 - val_loss: 0.6199 - val_accuracy: 0.7496
Epoch 2/100
5000/5000 [==============================] - 2s 328us/sample - loss: 0.5888 - accuracy: 0.7654 - val_loss: 0.5592 - val_accuracy: 0.7700
Epoch 3/100
5000/5000 [==============================] - 2s 328us/sample - loss: 0.5217 - accuracy: 0.8026 - val_loss: 0.5047 - val_accuracy: 0.7952
Epoch 4/100
5000/5000 [==============================] - 2s 329us/sample - loss: 0.4613 - accuracy: 0.8294 - val_loss: 0.4550 - val_accuracy: 0.8202
Epoch 5/100
5000/5000 [==============================] - 2s 331us/sample - loss: 0.4083 - accuracy: 0.8530 - val_loss: 0.4340 - val_accuracy: 0.8132
Epoch 6/100
5000/5000 [==============================] - 2s 327us/sample - loss: 0.3657 - accuracy: 0.8700 - val_loss: 0.3912 - val_accuracy: 0.8426
Epoch 7/100
5000/5000 [==============================] - 2

In [15]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

predictions = model.predict_classes(norm_test_texts, batch_size=512, verbose=0).ravel()

print("Accuracy: %.2f%%" % (accuracy_score(test_sentiments, predictions)*100))
print(classification_report(test_sentiments, predictions))
pd.DataFrame(confusion_matrix(test_sentiments, predictions))

Accuracy: 86.52%
             precision    recall  f1-score   support

          0       0.87      0.86      0.86     20028
          1       0.86      0.87      0.87     19972

avg / total       0.87      0.87      0.87     40000



,0,1
0,17201,2827
1,2567,17405
